# Анализ нормализации признаков, извлечённых с помощью BrepExtractor

В этом ноутбуке мы исследуем, насколько хорошо нормализованы признаки, получаемые из 3D-моделей с помощью BrepExtractor. Будут рассмотрены распределения, статистики и возможные выбросы для каждого блока признаков.

## 1. Импорт необходимых библиотек

Импортируем основные библиотеки для анализа и визуализации, а также классы BrepExtractor и DataModel.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import notebook_setup
from src.dataset import DatasetIO
from src.config import INTERIM_DATA_DIR
from src.features.brep import BrepExtractor
from src.dataset import DataModel


# Настройки визуализации
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 5)

pkl_file = INTERIM_DATA_DIR / "dataset_metadata.pkl"
dataset = DatasetIO.load_dataset_pickle(pkl_file)

print(f"Загружено объектов: {len(dataset)}")

2025-08-14 15:07:14.044 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: D:\workspace\projects\freelance\3d_recognition_analisis


Project root added to path: d:\workspace\projects\freelance\3d_recognition_analisis
2025-08-14 15:07:18.186 | INFO     | src.dataset:load_dataset_pickle:332 - Загрузка датасета из D:\workspace\projects\freelance\3d_recognition_analisis\data\interim\dataset_metadata.pkl
2025-08-14 15:07:18.190 | SUCCESS  | src.dataset:load_dataset_pickle:337 - Датасет загружен из D:\workspace\projects\freelance\3d_recognition_analisis\data\interim\dataset_metadata.pkl
Загружено объектов: 129


## 2. Загрузка и извлечение признаков из моделей

Загрузим список моделей, применим BrepExtractor для получения матриц признаков, соберём их в DataFrame для дальнейшего анализа.

In [2]:

extractor = BrepExtractor(output_mode='pooled')

feature_vectors = []
labels = []
model_ids = []

for item in dataset:
    fv = extractor.extract_single(item)
    if fv is not None:
        feature_vectors.append(fv.vector)
        labels.append(fv.label)
        model_ids.append(fv.model_id)
    else:
        # Если не удалось извлечь признаки, пропускаем объект
        print(f"Пропуск: {item.model_id}")

X = np.stack(feature_vectors)
y = np.array(labels)



2025-08-14 15:07:20.722 | INFO     | src.features.extractor:__init__:54 - Инициализация BRep экстрактора признаков на устройстве cpu
2025-08-14 15:07:20.723 | INFO     | src.features.brep:__init__:61 - Инициализация 'BRep' (K=256, LBO=16, bins=64, D=256, mode=pooled)
2025-08-14 15:07:20.806 | DEBUG    | src.features.brep:extract_single:108 - Матрица признаков 42. Ejector-01: (7, 256)
2025-08-14 15:07:20.887 | DEBUG    | src.features.brep:extract_single:108 - Матрица признаков 42. Ejector-02: (7, 256)
2025-08-14 15:07:20.966 | DEBUG    | src.features.brep:extract_single:108 - Матрица признаков 42. Ejector-03: (7, 256)
2025-08-14 15:07:21.107 | DEBUG    | src.features.brep:extract_single:108 - Матрица признаков 42. Ejector-04: (7, 256)
2025-08-14 15:07:21.169 | DEBUG    | src.features.brep:extract_single:108 - Матрица признаков 42. Ejector-05: (7, 256)
2025-08-14 15:07:21.246 | DEBUG    | src.features.brep:extract_single:108 - Матрица признаков 42. Ejector-06: (7, 256)
2025-08-14 15:07:2

In [3]:
from src.modeling.models.resnet1d import setup_logger, set_seed, TrainConfig, train_pooled


setup_logger("logs")
set_seed(42)
cfg = TrainConfig()
model, le, hist = train_pooled(X, y, cfg)




2025-08-14 15:07:58 | INFO     | Устройство: cpu
2025-08-14 15:07:58 | INFO     | Train size: 93, Val size: 36


Эпохи:   0%|          | 0/100 [00:00<?, ?эпоха/s]

Тренировка 1/100:   0%|          | 0/2 [00:00<?, ?it/s]

Валидация:   0%|          | 0/1 [00:00<?, ?it/s]

2025-08-14 15:07:59 | INFO     | [001/100] train_loss=2.7311 | train_acc=0.0968 | val_loss=2.4752 | val_acc=0.1667 | lr=3.00e-04
2025-08-14 15:07:59 | SUCCESS  | Новый лучший val_loss=2.4752. Сохранён чекпоинт → best_pooled_mlp.pt


...

Тренировка 50/100:   0%|          | 0/2 [00:00<?, ?it/s]

Валидация:   0%|          | 0/1 [00:00<?, ?it/s]

2025-08-14 15:08:04 | INFO     | [050/100] train_loss=0.6033 | train_acc=1.0000 | val_loss=1.0301 | val_acc=0.8333 | lr=6.00e-05
2025-08-14 15:08:04 | WARNING  | 🛑 Early stopping: терпение исчерпано.
2025-08-14 15:08:04 | INFO     | История обучения сохранена → history_pooled.json
2025-08-14 15:08:04 | INFO     | 📦 Загружен лучший чекпоинт эпохи 40 (val_loss=0.9863)


Финальная валидация:   0%|          | 0/1 [00:00<?, ?it/s]

2025-08-14 15:08:04 | INFO     | 
📊 ФИНАЛЬНЫЙ ОТЧЁТ
              precision    recall  f1-score   support

           0       1.00      0.33      0.50         3
           1       0.67      0.67      0.67         3
           2       0.75      1.00      0.86         3
           3       0.75      1.00      0.86         3
           4       1.00      1.00      1.00         3
           5       1.00      0.33      0.50         3
           6       0.50      0.67      0.57         3
           7       1.00      1.00      1.00         3
           8       1.00      1.00      1.00         3
           9       1.00      1.00      1.00         3
          10       1.00      1.00      1.00         3
          11       0.75      1.00      0.86         3

    accuracy                           0.83        36
   macro avg       0.87      0.83      0.82        36
weighted avg       0.87      0.83      0.82        36

2025-08-14 15:08:04 | INFO     | Accuracy: 0.8333
2025-08-14 15:08:04 | INFO     